In [1]:
import argparse
from bertviz import model_view
import jsonlines
import os
import numpy as np
import pytorch_lightning as pl

from constants import ENTITY_START_MARKER, ENTITY_END_MARKER
from data_loader import  DrugSynergyDataModule, make_fixed_length
from model import RelationExtractor, load_model
from preprocess import create_dataset
from utils import construct_row_id_idx_mapping, set_seed, write_error_analysis_file

from streamlit_single_relation_app import classify_message

/home/vijay/miniconda3/envs/synergy/lib/python3.8/site-packages/pytorch_lightning/metrics/__init__.py:43: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  rank_zero_deprecation(


In [2]:
checkpoint_path = "/home/vijay/drug-synergy-models/checkpoints_with_relation_pretraining_seed_2024_multiclass"

In [3]:
model, tokenizer, metadata = load_model(checkpoint_path, output_attentions=True)
tokenizer.add_tokens([ENTITY_START_MARKER, ENTITY_END_MARKER])


2

In [ ]:
tokenizer.add_tokens([ENTITY_START_MARKER, ENTITY_END_MARKER])
drugs = open("drugs.txt").read().lower().split()
tokenizer.add_tokens(drugs)

In [4]:
'''
message_text = "The aims of this study were to determine the effects of (a) combining the epidermal growth factor"+ \
" receptor (EGFR) blocker (erlotinib) and the cyclooxygenase-2 inhibitor (celecoxib) on cell growth and apoptosis"+ \
" in human pancreatic cancer cell lines, (b) baseline EGFR expression on the potentiation of erlotinib-induced"+ \
" apoptosis by celecoxib, and (c) the effects of the combination on the expression of the COX-2, EGFR, HER-2/neu,"+ \
" and nuclear factor-kappaB (NF-kappaB). Baseline expression of EGFR was determined by Western blot analysis in"+ \
" five human pancreatic cancer cell lines. BxPC-3, PANC-1, and HPAC had high EGFR and MIAPaCa had low EGFR. Cells"+ \
" were grown in culture and treated with erlotinib (1 and 10 micromol/L), celecoxib (1 and 10 micromol/L), and the"+ \
" combination. Growth inhibition was evaluated using 3-(4,5-dimethylthiazol-2-yl)-2,5-diphenyltetrazolium bromide"+ \
" assay, and apoptosis was assayed by ELISA. Reverse transcriptase-PCR was used to evaluate COX-2 and EGFR mRNA."+ \
" EGFR, COX-2, and HER-2/neu expression was determined by Western immunoblotting. Electrophoretic mobility shift"+ \
" assay was used to evaluate NF-kappaB activation. Growth inhibition and apoptosis were significantly ( P < 0.05 )"+ \
" higher in BxPC-3 , HPAC , and PANC-1 cells treated with <<m>> celecoxib <</m>> and <<m>> erlotinib <</m>> than cells "+ \
" treated with either"+ \
" celecoxib or erlotinib or cisplatin . However, no potentiation in growth inhibition or apoptosis was observed in the MIAPaCa"+ \
" cell line with low expression of the EGFR. Significant down-regulation of COX-2 and EGFR expression was observed"+ \
" in the BxPC-3 and HPAC cells treated with the combination of erlotinib (1 micromol/L) and celecoxib (10 micromol/L)"+ \
" compared with celecoxib- or erlotinib-treated cells. celecoxib significantly down-regulated HER-2/neu expression in"+ \
" BxPC-3 and HPAC cell lines. Significant inhibition of NF-kappaB activation was observed in BxPC-3 and HPAC cell"+ \
" lines treated with erlotinib and celecoxib. (a) celecoxib can potentiate erlotinib-induced growth inhibition and"+ \
" apoptosis in pancreatic cell lines, (b) high baseline EGFR expression is a predictor of this potentiation, and (c)"+ \
" the down-regulation of EGFR, COX-2, and HER-2/neu expression and NF-kappaB inactivation contributes to the"+ \
" potentiation of erlotinib by celecoxib, suggesting an alternative to cisplatin."
'''

message_text = "Various cutaneous side-effects have been reported with anti-melanoma systemic therapies. "+ \
"This study investigated the changes in melanocytic lesion pigmentation in patients on four different therapies."+ \
" ### methods We analysed the serial dermatoscopic photographs of atypical melanocytic lesions taken from patients"+ \
" with advanced metastatic melanoma on four different systemic therapies ( selective BRAF-inhibitor monotherapy ,"+ \
" <<m>> dabrafenib <</m>> combined with <<m>> trametinib <</m>> [ D&T ] , anti-programmed cell death protein 1 [ "+ \
"anti-PD1 ] therapies , and anti-PD1 combined with ipilimumab ) seen from February 2013 to May 2016 . We compared "+ \
"these changes with the melanocytic lesions of 10 control patients. ### results In the control group, 19% of naevi "+ \
"lightened, 64% did not change and 17% darkened. Only the BRAF inhibitor group showed more darkened lesions than "+ \
"controls (37%, P < 0.001). Meanwhile, there were more lightened naevi in the D&T therapy group (86%, P < 0.001) "+ \
"as well as the anti-PD1 and ipilimumab groups (59%, P < 0.001) than controls. Patients on anti-PD1 monotherapy "+ \
"had more lightened (49%) and fewer darkened naevi (9%) than controls, but differences were not significant. ###"+ \
" conclusions Our study showed that different anti-melanoma systemic therapies have different effects on the "+ \
"pigmentation of melanocytic lesions. BRAF inhibitor may have the propensity to cause darkening while D&T therapy"+ \
" and anti-PD1 caused lightening compared with controls. The findings emphasise the importance of regular "+ \
"dermatological monitoring in specialised clinics for patients on anti-melanoma systemic therapy. Clinicians "+ \
"should expect changes in the global pigmentation of melanocytic lesions but be suspicious of lesions with "+ \
"structural changes."

In [5]:
inputs = tokenizer.encode(message_text, return_tensors='pt')
tokens = tokenizer.convert_ids_to_tokens(inputs[0]) 

In [ ]:
#tokens = make_fixed_length(tokens, metadata.max_seq_length, padding_value = "<PAD>")

In [6]:
model_output, attention = classify_message(message_text, model, tokenizer, metadata.max_seq_length)

/home/vijay/drug-synergy-models/streamlit_single_relation_app.py:63: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probabilities = torch.nn.functional.softmax(logits)[0]


In [7]:
def only_keep_nth_head(attention_layer, head_idxs, seq_length):
    return attention_layer[:, head_idxs, :seq_length, :seq_length]

In [8]:
attention_pruned = [only_keep_nth_head(attention[i], list(range(12)), len(tokens)) for i in range(12)]

In [9]:
def print_top_attended_words(attention_tensors, query_token_idx, layer_idx, attn_head_idx, num_words_to_show=10):
    top_self_attentions = np.argsort(-attention_tensors[layer_idx][0][attn_head_idx][query_token_idx].detach())[:num_words_to_show]
    for other_token_idx in top_self_attentions:
        word = f"{tokens[other_token_idx]} ({other_token_idx})"
        if len(word) <= 7:
            num_tabs = 4
        elif len(word) >= 16:
            num_tabs = 2
        else:
            num_tabs = 3
        tabs = "".join(["\t" for _ in range(num_tabs)])
        print(f"{tokens[other_token_idx]} ({other_token_idx}){tabs}{attention_pruned[layer_idx][0][attn_head_idx][start][other_token_idx]}")

TRANSFORMER_LAYER_IDX = 0
ATTENTION_HEAD_IDX = 4
start_token_idxs = [i for i, v in enumerate(tokens) if v == "<<m>>"]
for attention_head in range(12):
    start = start_token_idxs[0]
    print(f"Showing attention head {attention_head} in transformer layer {TRANSFORMER_LAYER_IDX}")
    print(f"Query entity: entity marker before \"{tokens[start+1]}\" ({start})")
    print("===================================================")
    print(f"Subword\t\t\t|\tAttention weight\n---------------------------------------------------")
    print_top_attended_words(attention_pruned, start, TRANSFORMER_LAYER_IDX, attention_head)
    print("\n\n")

Showing attention head 0 in transformer layer 0
Query entity: entity marker before "dab" (69)
Subword			|	Attention weight
---------------------------------------------------
emphasis (338)			0.014879440888762474
clinics (351)			0.014471146278083324
study (288)			0.014462144114077091
special (349)			0.010471430607140064
changes (127)			0.010239566676318645
therapies (296)			0.009719575755298138
systemic (295)			0.009239290840923786
[CLS] (0)			0.00855463556945324
dermatol (344)			0.008067985996603966
showed (289)			0.00772210443392396



Showing attention head 1 in transformer layer 0
Query entity: entity marker before "dab" (69)
Subword			|	Attention weight
---------------------------------------------------
fewer (264)			0.09122643619775772
propensity (314)		0.026614848524332047
conclusions (286)		0.02536991611123085
monotherapy (67)		0.020404068753123283
monotherapy (254)		0.020290911197662354
serial (41)			0.018741831183433533
##inib (80)			0.016001060605049133
[SEP] (383)			0.0158

# model_view(attention_pruned, tokens[:metadata.max_seq_length])